In [5]:
import os, sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.api import VAR
import pandas as pd
from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

In [ ]:
data = pd.read_csv("../data/BrentOilPrices.csv", parse_dates=['Date'])

In [ ]:
# Prepare multivariate data
data_multivariate = data[['Price', 'GDP', 'Inflation', 'Exchange_Rate']]  # Replace with actual columns
data_multivariate = data_multivariate.dropna()

# Fit VAR model
model_var = VAR(data_multivariate)
results_var = model_var.fit(maxlags=5)

# Forecast
lag_order = results_var.k_ar
forecast_var = results_var.forecast(data_multivariate.values[-lag_order:], steps=12)
print(forecast_var)

In [ ]:
# Fit Markov-Switching ARIMA
model_ms = MarkovAutoregression(data['Price'], k_regimes=2, order=(1, 1, 1))
results_ms = model_ms.fit()

# Print summary
print(results_ms.summary())

In [ ]:
# Prepare data for LSTM
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

X, y = create_dataset(data['Price'].values)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(50, input_shape=(X.shape[1], 1)))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model_lstm.fit(X, y, epochs=100, batch_size=32, verbose=2)